任务一、分析各销售机商品销售数据，总结规律，给出每台销售机饮料类商品的标签。

In [1]:
import numpy as np
import pandas as pd

In [2]:
order_A = pd.read_csv('task1-1A.csv',encoding = 'gbk')
category = pd.read_csv('data_2.csv',encoding = 'gbk')

In [3]:
#筛选饮料类商品
order_A_cate = pd.merge(order_A,category)
order_A_drink = order_A_cate[order_A_cate['大类'] == '饮料']
order_A_drink.head()

,订单号,设备ID,应付金额,实际金额,商品,支付时间,地点,状态,提现,大类,二级类
228,DD201708167493683503095993712,E43A6E078A04172,3.0,3.0,250ml维他柠檬茶,2017/1/1 19:55,A,已出货未退款,已提现,饮料,茶饮料
229,DD201708167493507701935975302,E43A6E078A04172,3.0,3.0,250ml维他柠檬茶,2017/1/1 20:25,A,已出货未退款,已提现,饮料,茶饮料
230,DD201708167493662547010881037,E43A6E078A04172,3.0,3.0,250ml维他柠檬茶,2017/1/3 9:50,A,已出货未退款,已提现,饮料,茶饮料
231,DD20170816749300338025192860536,E43A6E078A04172,6.0,6.0,250ml维他柠檬茶,2017/1/4 0:29,A,已出货未退款,已提现,饮料,茶饮料
232,DD201708167493849176273799289,E43A6E078A04172,3.0,3.0,250ml维他柠檬茶,2017/1/8 21:06,A,已出货未退款,已提现,饮料,茶饮料


In [4]:
group_order_A_drink = order_A_drink[['订单号','商品']].groupby(by = '商品')
#将饮料商品按销量降序排列
group_order_A_selling = group_order_A_drink.count().sort_values(by = '订单号',ascending = False)
group_order_A_selling.columns = ['总销量']
group_order_A_selling.head()

,总销量
商品,
怡宝纯净水,531
东鹏特饮,448
阿萨姆奶茶,326
脉动,309
营养快线,255


In [5]:
#划分商品滞销/正常/热销，要根据商品在这台销售机的分布，是针对于在此销售机上各商品的相对而言
#因此可将各商品依据在这台销售机的销量这一特征，看作是仅有一个特征的聚类，分为3类
from sklearn.cluster import KMeans
kmModel=KMeans(n_clusters=3)
amounts_A = np.array(group_order_A_selling['总销量']).reshape(-1,1)
kmModel=kmModel.fit(amounts_A)
pTarget=kmModel.predict(amounts_A)
print('分类结果:',pTarget)

#找到销售机A的判别热销/正常/滞销的阈值点
clabel = 0
lv1_A = 0
lv2_A = 0
for k in range(len(pTarget)):
    if k == 0:
        continue
    if pTarget[k] != pTarget[k-1]:
        if clabel == 0:
            lv1_A = amounts_A[k][0]
            print('A销售机的热销/正常阈值点:',lv1_A)
            clabel =  1
        elif clabel == 1:
            lv2_A = amounts_A[k][0]
            print('A销售机的正常/滞销预置点:',lv2_A)

分类结果: [1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
A销售机的热销/正常阈值点: 255
A销售机的正常/滞销预置点: 78


In [6]:
group_order_A_selling['标签'] = '滞销'
for k in range(len(group_order_A_selling)):
    if group_order_A_selling['总销量'][k] > lv1_A:
        group_order_A_selling['标签'][k] = '热销'
    elif group_order_A_selling['总销量'][k] > lv2_A:
        group_order_A_selling['标签'][k] = '正常'
    else:
        group_order_A_selling['标签'][k] = '滞销'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
#销售机A的标签
A_labels = pd.DataFrame(np.arange(len(group_order_A_selling)) + 1,columns = ['序号'])
A_labels['饮料类商品'] = group_order_A_selling.index
A_labels['标签'] = group_order_A_selling['标签'].tolist()
A_labels.to_csv('task3-1A.csv',index = False,encoding = 'gbk')

In [8]:
A_labels.head(10)

,序号,饮料类商品,标签
0,1,怡宝纯净水,热销
1,2,东鹏特饮,热销
2,3,阿萨姆奶茶,热销
3,4,脉动,热销
4,5,营养快线,正常
5,6,统一冰红茶,正常
6,7,250ml维他柠檬茶,正常
7,8,250ml维他原味豆奶,正常
8,9,红牛,正常
9,10,统一绿茶,正常


In [9]:
def create_labels(mac):
    """
    输入：销售机台号：'A','B','C','D','E'
    制作对应销售机标签
    """
    print('绘制%c销售机的标签...'%mac)
    category = pd.read_csv('data_2.csv',encoding = 'gbk')
    exec_str = "pd.read_csv('task1-1%c.csv',encoding = 'gbk')"%mac
    order = eval(exec_str)
    order_cate = pd.merge(order,category)
    order_drink = order_cate[order_cate['大类'] == '饮料']
    group_order_drink = order_drink[['订单号','商品']].groupby(by = '商品')
    group_order_selling = group_order_drink.count().sort_values(by = '订单号',ascending = False)
    group_order_selling.columns = ['总销量']
    
    kmModel=KMeans(n_clusters=3)
    amounts = np.array(group_order_selling['总销量']).reshape(-1,1)
    kmModel=kmModel.fit(amounts)
    pTarget=kmModel.predict(amounts)
    
    clabel = 0
    lv1 = 0
    lv2 = 0
    for k in range(len(pTarget)):
        if k == 0:
            continue
        if pTarget[k] != pTarget[k-1]:
            if clabel == 0:
                lv1 = amounts[k][0]
                print('%c销售机的热销/正常阈值点:'%mac,lv1)
                clabel =  1
            elif clabel == 1:
                lv2 = amounts[k][0]
                print('%c销售机的正常/滞销预置点:'%mac,lv2)
    group_order_selling['标签'] = '滞销'
    for k in range(len(group_order_selling)):
        if group_order_selling['总销量'][k] > lv1:
            group_order_selling['标签'][k] = '热销'
        elif group_order_selling['总销量'][k] > lv2:
            group_order_selling['标签'][k] = '正常'
        else:
            group_order_selling['标签'][k] = '滞销'
        
    labels = pd.DataFrame(np.arange(len(group_order_selling)) + 1,columns = ['序号'])
    labels['饮料类商品'] = group_order_selling.index
    labels['标签'] = group_order_selling['标签'].tolist()
    labels.to_csv('task3-1%c.csv'%mac,index = False,encoding = 'gbk')
    print('%c销售机的标签绘制完成。'%mac)
    print()
    

In [10]:
create_labels('B')

绘制B销售机的标签...
B销售机的热销/正常阈值点: 488
B销售机的正常/滞销预置点: 139


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


B销售机的标签绘制完成。



In [11]:
machine = ['A','B','C','D','E']
for k in machine:
    create_labels(k)

绘制A销售机的标签...
A销售机的热销/正常阈值点: 255
A销售机的正常/滞销预置点: 78


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


A销售机的标签绘制完成。

绘制B销售机的标签...
B销售机的热销/正常阈值点: 488
B销售机的正常/滞销预置点: 139
B销售机的标签绘制完成。

绘制C销售机的标签...
C销售机的热销/正常阈值点: 525
C销售机的正常/滞销预置点: 166
C销售机的标签绘制完成。

绘制D销售机的标签...
D销售机的热销/正常阈值点: 188
D销售机的正常/滞销预置点: 68
D销售机的标签绘制完成。

绘制E销售机的标签...
E销售机的热销/正常阈值点: 996
E销售机的正常/滞销预置点: 309
E销售机的标签绘制完成。



任务二、在任务一的基础上进行标签拓展，依据标签生成完整的销售机图像。

In [12]:
import jieba
from wordcloud import WordCloud
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [41]:
def create_port(mac):
    print('绘制销售机%c的画像...'%mac)
    label = eval("pd.read_csv('task3-1%c.csv',encoding = 'gbk')"%mac)
    #读取该销售机热销及正常销售的商品
    proper_sell = label[label.标签.isin(['热销','正常'])]
    proper_sell_drink = proper_sell['饮料类商品'].tolist()
    f = open('normal_sell_%c.txt'%mac,'w')
    for k in proper_sell_drink:
        f.write(k+' ')
    f.close()
    #绘制关于该销售机非滞销销售商品的词云图
    f = open('normal_sell_%c.txt'%mac,'r').read()
    font = 'C:\Windows\Fonts\FZSTK.TTF'
    cut = jieba.cut(f)
    cut_content = ' '.join(cut)
    img = Image.open('o_002.jpg') # 以什么图片进行显示
    img_array = np.array(img) # 将图片转换为数组
    wc = WordCloud(
        background_color='white',
        mask=img_array, # 若没有该项，则生成默认图片
        font_path=font # 中文分词必须有中文字体设置
    )
    wc.generate_from_text(cut_content) # 绘制图片
    wc.to_file('task3-2%c.png'%mac) # 保存图片
    print('销售机%c的画像绘制完成。'%mac)
    print()

In [42]:
mac_lists = ['A','B','C','D','E']
for mac in mac_lists:
    create_port(mac)

绘制销售机A的画像...
销售机A的画像绘制完成。

绘制销售机B的画像...
销售机B的画像绘制完成。

绘制销售机C的画像...
销售机C的画像绘制完成。

绘制销售机D的画像...
销售机D的画像绘制完成。

绘制销售机E的画像...
销售机E的画像绘制完成。

